# Laplacian deformation

Using Laplacian deformation for mesh and animation.

Notebook by Jerome Eippers, 2025

In [ ]:
%matplotlib widget
import pickle
import numpy as np
import scipy
import scipy.sparse as sp
import scipy.sparse.linalg as spla

from ipywidgets import widgets, interact
import ipyanimlab as lab

viewer = lab.Viewer(move_speed=5, width=1280, height=720)

## The mesh
In our case a mesh is a set of **vertices** $ v_i $ in $ R^3$ (points in 3D space), and **edges** (connections between vertices), treating the mesh as a graph.



## Laplacian Matrix
The Laplacian matrix $ L $ is a key tool in spectral graph theory and geometry processing. For a mesh, it encodes the connectivity of the vertices and is defined as:

$$
L = D - A
$$

- $ A $ is the **adjacency matrix**, where $ A_{ij} = 1 $ if there’s an edge between vertex $ i $ and $ j $, and $ 0 $ otherwise.
- $ D $ is the **degree matrix**, a diagonal matrix where $ D_{ii} $ is the number of edges connected to vertex $ i $.


In [ ]:
def compute_degree_matrix(Adjacency):
    # create the degree matrix from the Adjacency matrix
    
    Degree = sp.lil_matrix(Adjacency.shape)
    for i in range(Adjacency.shape[0]):
        Degree[i,i] = Adjacency[i, :].sum()
    return Degree
    

def generate_laplacian(count, strip_len, strip_width):

    # create vertices
    vertices = np.zeros([count, 2, 3], dtype=np.float32)
    vertices[:, :, 2] = np.linspace(0, strip_len, count)[:, np.newaxis]
    vertices[:, 1, 0] = strip_width
    vertices = vertices.reshape(-1, 3)

    # compute adjancency of each vertices
    Adjacency = sp.lil_matrix((count*2, count*2))

    # connect vertices
    for i in range(0, count*2-1, 2):
        Adjacency[i, i+1] = 1
        Adjacency[i+1, i] = 1
        
    for i in range(0, count*2-2, 2):
        Adjacency[i, i+2] = 1
        Adjacency[i+2, i] = 1

    for i in range(1, count*2-2, 2):
        Adjacency[i, i+2] = 1
        Adjacency[i+2, i] = 1

    # compute the degree
    Degree = compute_degree_matrix(Adjacency)

    # finally the laplacian
    L = Degree - Adjacency
    return L.tocsc(), Adjacency.tocsc(), vertices

In [ ]:
def draw(Adjacency, vertices=None, new_vertices=None, edges=None, final_draw=True):
    if final_draw:
        viewer.begin_shadow()
        viewer.end_shadow()
        
        viewer.begin_display()
        viewer.draw_ground()  
        viewer.end_display()
        
        viewer.disable(depth_test=True)
    
    buf_size = 512
    if vertices is not None:
        v = np.zeros([buf_size,3], dtype=np.float32)
        vi = 0
        horizontal, vertical = sp.triu(Adjacency).nonzero()
        for i, j in zip(horizontal, vertical):
            v[vi] = vertices[i]
            v[vi+1] = vertices[j]
            vi += 2
            if vi >= buf_size:
                viewer.draw_lines(v)
                v = np.zeros([buf_size,3], dtype=np.float32)
                vi = 0
        viewer.draw_lines(v)

    if new_vertices is not None:
        v = np.zeros([buf_size,3], dtype=np.float32)
        vi = 0
        horizontal, vertical = sp.triu(Adjacency).nonzero()
        for i, j in zip(horizontal, vertical):
            v[vi] = new_vertices[i]
            v[vi+1] = new_vertices[j]
            vi += 2
            if vi >= buf_size:
                viewer.draw_lines(v, color=np.array([1,1,0], dtype=np.float32))
                v = np.zeros([buf_size,3], dtype=np.float32)
                vi = 0
        viewer.draw_lines(v, color=np.array([1,1,0], dtype=np.float32))

    if edges is not None:
        v = np.zeros([buf_size,3], dtype=np.float32)
        vi = 0
        for i, j in edges:
            v[vi] = new_vertices[i]
            v[vi+1] = new_vertices[j]
            vi += 2
            if vi >= buf_size:
                viewer.draw_lines(v, color=np.array([1,0,0], dtype=np.float32))
                v = np.zeros([buf_size,3], dtype=np.float32)
                vi = 0
        viewer.draw_lines(v, color=np.array([1,0,0], dtype=np.float32))

    if final_draw:
        viewer.execute_commands()
    return viewer

In [ ]:
_, Adjacency, vertices = generate_laplacian(15, 200, 20)
draw(Adjacency, vertices)

## Laplacian Coordinates
The Laplacian matrix is used to compute **Laplacian coordinates**, which represent the local geometry of the mesh. For each vertex $ v_i $, its Laplacian coordinate $ \Delta v_i $ is:

$$
\Delta v_i = \mathbf{v}_i - \frac{1}{d_i} \sum_{j \in N(i)} \mathbf{v}_j
$$

- $ N(i) $ is the set of neighbors of $ v_i $.
- $ d_i $ is the degree of $ v_i $ (number of neighbors).

In matrix form, this can be written as:

$$
\mathbf{\Delta} = L\mathbf{V}
$$

where $ \mathbf{V} $ is the $ n \times 3 $ matrix of vertex positions, and $ \mathbf{\Delta} $ is the $ n \times 3 $ matrix of Laplacian coordinates.



## Mesh Deformation

When deforming the mesh, we want to preserve the local structure while moving some control points to desired positions. The goal is to minimize the difference between the original and deformed Laplacian coordinates.

### Formulating the Error Functional
The error functional is formulated as:

$$
E(V') = \sum_{i=1}^n \| \Delta v'_i - \Delta v_i \|^2
$$


where $ \mathbf{c}_j $ are the target positions for the handles and fixed vertices.

This is a **least-squares optimization problem** and can be solved using linear algebra techniques.


### Least Squares Minimization

To minimize $E(V')$ we rewrite it in matrix form:

$$
\min_{V'} \| LV' - LV \|^2
$$

Taking the gradient and setting it to zero results in the linear system:

$$
LV' = LV
$$

which is a **sparse system of equations** because $ L $ is sparse.


In [ ]:
L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

A = L
b = delta

new_vertices = np.zeros_like(vertices)
new_vertices[:, 0] = spla.lsqr(A, b[:,0])[0]
new_vertices[:, 1] = spla.lsqr(A, b[:,1])[0]
new_vertices[:, 2] = spla.lsqr(A, b[:,2])[0]

draw(Adjacency, vertices, new_vertices)

## Add anchors

We will add at anchors that will force some of the vertices to be at specifics positions

$$
\begin{bmatrix}
L \\
C
\end{bmatrix}
\begin{bmatrix}
V \\
\end{bmatrix}
=
\begin{bmatrix}
b \\
d
\end{bmatrix}
$$


In [ ]:
L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

A_anchor = sp.lil_matrix((2, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1

d_anchors = np.array([[0,0,0], [0,0,200]])

A = sp.vstack([L, A_anchor], format='csc')
rhs = np.concatenate([delta, d_anchors])

new_vertices = np.zeros_like(vertices)
new_vertices[:, 0] = spla.lsqr(A, rhs[:,0])[0]
new_vertices[:, 1] = spla.lsqr(A, rhs[:,1])[0]
new_vertices[:, 2] = spla.lsqr(A, rhs[:,2])[0]

draw(Adjacency, vertices, new_vertices)

In [ ]:
L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

A_anchor = sp.lil_matrix((3, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1
A_anchor[2, 12] = 1

d_anchors = np.array([[0,0,0], [0,0,200], [50,0,120]])

A = sp.vstack([L, A_anchor], format='csc')
rhs = np.concatenate([delta, d_anchors])

new_vertices = np.zeros_like(vertices)
new_vertices[:, 0] = spla.lsqr(A, rhs[:,0])[0]
new_vertices[:, 1] = spla.lsqr(A, rhs[:,1])[0]
new_vertices[:, 2] = spla.lsqr(A, rhs[:,2])[0]

draw(Adjacency, vertices, new_vertices)

## Pseudoinverse in Least Squares Problems

When solving a system of linear equations of the form:
$$ A \mathbf{x} = \mathbf{b} $$
in cases where $ A $ is not square or does not have full rank, we often seek a **least squares solution**, which minimizes the squared error:
$$ \min_\mathbf{x} \| A\mathbf{x} - \mathbf{b} \|^2. $$
This is particularly useful when the system is overdetermined (more equations than unknowns) or underdetermined (fewer equations than unknowns).

### Normal Equations and the Pseudoinverse
The standard approach to solving least squares problems is via the **normal equations**:
$$ A^T A \mathbf{x} = A^T \mathbf{b}. $$
If $ A^T A $ is invertible, the unique solution is:
$$ \mathbf{x} = (A^T A)^{-1} A^T \mathbf{b}. $$
However, if $ A^T A $ is singular or nearly singular, direct inversion is not numerically stable.

In [ ]:
L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

A_anchor = sp.lil_matrix((3, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1
A_anchor[2, 12] = 1

d_anchors = np.array([[0,0,0], [0,0,200], [50,0,120]])

A = sp.vstack([L, A_anchor], format='csc')
rhs = np.concatenate([delta, d_anchors])

ATA = A.transpose().dot(A)
rhs_x = A.transpose().dot(rhs[:, 0])
rhs_y = A.transpose().dot(rhs[:, 1])
rhs_z = A.transpose().dot(rhs[:, 2])

new_vertices = np.zeros_like(vertices)
new_vertices[:, 0] = spla.spsolve(ATA, rhs_x)
new_vertices[:, 1] = spla.spsolve(ATA, rhs_y)
new_vertices[:, 2] = spla.spsolve(ATA, rhs_z)

draw(Adjacency, vertices, new_vertices)

### The Moore-Penrose Pseudoinverse
The **Moore-Penrose pseudoinverse** of $ A $, denoted $ A^+ $, provides a general and stable way to compute the least squares solution:
$$ \mathbf{x} = A^+ \mathbf{b}. $$
The pseudoinverse is defined as the unique matrix that satisfies the following properties:
1. $ A A^+ A = A $
2. $ A^+ A A^+ = A^+ $
3. $ (A A^+)^T = A A^+ $
4. $ (A^+ A)^T = A^+ A $

### Computation via Singular Value Decomposition (SVD)
A numerically stable way to compute $ A^+ $ is through **Singular Value Decomposition (SVD)**:
$$ A = U \Sigma V^T, $$
where $ U $ and $ V $ are orthogonal matrices, and $ \Sigma $ is a diagonal matrix containing the singular values. The pseudoinverse is computed as:
$$ A^+ = V \Sigma^+ U^T, $$
where $ \Sigma^+ $ is obtained by inverting nonzero singular values and transposing the matrix.

### Why Use the Pseudoinverse?
- **Generalized solution**: It works even if $ A $ is singular or non-square.
- **Numerical stability**: SVD-based computation is more stable than inverting $ A^T A $.
- **Handles rank-deficient cases**: Provides the minimum-norm least squares solution when multiple solutions exist.

In [ ]:
L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

A_anchor = sp.lil_matrix((3, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1
A_anchor[2, 12] = 1

d_anchors = np.array([[0,0,0], [0,0,200], [50,0,120]])

A = sp.vstack([L, A_anchor], format='csc')
rhs = np.concatenate([delta, d_anchors])

new_vertices = scipy.linalg.pinv(A.todense()).dot(rhs)

draw(Adjacency, vertices, new_vertices)

## Lagrange Multipliers and KKT System

### Basic Problem

We want to solve the constrained optimization problem:

$$
\min_x \frac{1}{2} \|Ax - b\|^2 \quad \text{subject to} \quad Cx = d
$$

where:
- $ A \in \mathbb{R}^{m \times n} $, $ b \in \mathbb{R}^{m} $ define the least-squares objective.
- $ C \in \mathbb{R}^{p \times n} $, $ d \in \mathbb{R}^{p} $ define the constraints.

### Lagrangian Formulation

We introduce Lagrange multipliers $ \lambda \in \mathbb{R}^{p} $ and define the Lagrangian:

$$
\mathcal{L}(x, \lambda) = \frac{1}{2} \|Ax - b\|^2 + \lambda^T (Cx - d).
$$

### First-Order Optimality Conditions

Taking derivatives with respect to $ x $:

$$
\nabla_x \mathcal{L} = A^T(Ax - b) + C^T \lambda = 0.
$$

Taking derivatives with respect to $ \lambda $:

$$
\nabla_\lambda \mathcal{L} = Cx - d = 0.
$$

### KKT System

Rewriting the equations as a linear system:

$$
\begin{bmatrix}
A^T A & C^T \\
C & 0
\end{bmatrix}
\begin{bmatrix}
x \\
\lambda
\end{bmatrix}
=
\begin{bmatrix}
A^T b \\
d
\end{bmatrix}.
$$

This is known as the **Karush-Kuhn-Tucker (KKT) system**. Solving this system gives the optimal $ x $ and Lagrange multipliers $ \lambda $.


In [ ]:
L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

A_anchor = sp.lil_matrix((3, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1
A_anchor[2, 12] = 1

d_anchors = np.array([[0,0,0], [0,0,200], [50,0,120]])

LTL = L.transpose().dot(L)
zero_block = sp.csc_matrix((A_anchor.shape[0], A_anchor.shape[0]))

A = sp.bmat([
    [LTL, A_anchor.transpose()],
    [A_anchor, zero_block]
], format='csc')

new_vertices = np.zeros_like(vertices)
new_vertices[:, 0] = spla.spsolve(A, np.concatenate([L.transpose().dot(delta[:,0]), d_anchors[:,0]]))[:vertices.shape[0]]
new_vertices[:, 1] = spla.spsolve(A, np.concatenate([L.transpose().dot(delta[:,1]), d_anchors[:,1]]))[:vertices.shape[0]]
new_vertices[:, 2] = spla.spsolve(A, np.concatenate([L.transpose().dot(delta[:,2]), d_anchors[:,2]]))[:vertices.shape[0]]

draw(Adjacency, vertices, new_vertices)

## Extending the Laplacian System to 3D Using the Kronecker Product


In 3D, we extend the same system but now with coordinates $ V = (V_x, V_y, V_z) $. Instead of solving three separate least-squares problems, we use the **Kronecker product** to solve them all at once.

### Definition

The **Kronecker product** of $ L $ with the identity matrix $ I_3 $ (which accounts for $ x, y, z $) is:

$$
\tilde{L} = L \otimes I_3
$$

where:

- $ L \otimes I_3 $ is of size $ (3n \times 3n) $,
- $ I_3 $ is the $ 3 \times 3 $ identity matrix.

Using this, we rewrite the **3D system** as:

$$
\tilde{L} \tilde{V}' = \tilde{L} \tilde{V}.
$$

where $ \tilde{V} $ is the **flattened** version of $ V $:

$$
\tilde{V} = \begin{bmatrix} V_x \\ V_y \\ V_z \end{bmatrix} \quad \text{(size $ 3n \times 1 $)}
$$

This single equation now solves for all three coordinates simultaneously.


### Adding Soft Anchor Constraints

To **softly** enforce anchor constraints, we add a weighted least-squares penalty for certain vertices. 
This corresponds to adding extra rows to the system:

$$
\begin{bmatrix} \tilde{L} \\ w \tilde{C} \end{bmatrix} \tilde{V}'
=
\begin{bmatrix} \tilde{L} \tilde{V} \\ w \tilde{d} \end{bmatrix}.
$$

where:
- $ \tilde{C} $ is a selection matrix that picks anchor vertices,
- $ \tilde{d} $ contains target positions for the anchors,
- $ w $ is a **weight** controlling the softness of the constraint.



In [ ]:
anchor_weights = 3.0

L, Adjacency, vertices = generate_laplacian(15, 200, 20)

# I3 is the 3x3 identity matrix
I3 = sp.eye(3, format='csc')

L3D = sp.kron(L, I3, format='csc')

delta3D = L3D @ vertices.flatten()

A_anchor = sp.lil_matrix((3, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1
A_anchor[2, 12] = 1

A_anchor3D = sp.kron(A_anchor, I3, format='csc') * anchor_weights

d_anchors = np.array([[0,0,0], [0, 0, 200], [50, 0, 120]]) * anchor_weights

A = sp.vstack([L3D, A_anchor3D], format='csc')
rhs = np.concatenate([delta3D, d_anchors.flatten()])

new_vertices = spla.lsqr(A, rhs)[0].reshape(-1, 3)

draw(Adjacency, vertices, new_vertices)

In [ ]:
anchor_weights = 5.0

L, Adjacency, vertices = generate_laplacian(15, 200, 20)

# I3 is the 3x3 identity matrix
I3 = sp.eye(3, format='csc')

L3D = sp.kron(L, I3, format='csc')

delta_3D = L3D @ vertices.flatten()

A_anchor = sp.lil_matrix((4, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1
A_anchor[2, 12] = 1
A_anchor[3, 5] = 1

A_anchor3D = sp.kron(A_anchor, I3, format='csc') * anchor_weights

d_anchors = np.array([[0,0,0], [0, 0, 200], [50, 0, 120], [0, 0, 50]]) * anchor_weights

A = sp.vstack([L3D, A_anchor3D], format='csc')
rhs = np.concatenate([delta_3D, d_anchors.flatten()])

new_vertices = spla.lsqr(A, rhs)[0].reshape(-1, 3)

draw(Adjacency, vertices, new_vertices)

##  Edge Length Preservation

$$
\min_V \| HV - e(V) \|^2
$$

where:
- $ H $ is the **edge incidence matrix** (captures which vertices are connected).
- $ e(V) $ is a **nonlinear function of $ V $** enforcing edge length preservation.

Since $ e(V) $ depends on the **current** vertex positions, we cannot solve this as a simple $ Ax = b $ system. Instead, we **iteratively linearize** the problem using **Gauss-Newton**.


### Edge incidence matrix
$$
H_{(i,j), k} =
\begin{cases}
+1, & \text{if } k = i, \\
-1, & \text{if } k = j, \\
0,  & \text{otherwise.}
\end{cases}
$$

### Length function

The function $e(V)$ is designed to preserve the original edge lengths during deformation. Consider an edge $(i,j) \in E$ with:
- $ v_i $ and $ v_j $: the current positions of the vertices.
- $ \tilde{l}_{ij} $: the original (rest) length of the edge $(i,j)$.
- $ l_{ij} = \| v_i - v_j \| $: the current length of the edge after deformation.

The edge constraint function is defined as

$$
e(v_i, v_j) = \frac{\tilde{l}_{ij}}{l_{ij}} \, (v_i - v_j).
$$


In [ ]:
iteration_count = 10
anchor_weights = 5
length_weights = 5

edges = ((1,3),(3,5),(5,7),(7,9))

L, Adjacency, vertices = generate_laplacian(15, 200, 20)

# I3 is the 3x3 identity matrix
I3 = sp.eye(3, format='csc')

L3D = sp.kron(L, I3, format='csc')

delta_3D = L3D @ vertices.flatten()

A_anchor = sp.lil_matrix((4, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, -1] = 1
A_anchor[2, 12] = 1
A_anchor[3, 5] = 1

A_anchor3D = sp.kron(A_anchor, I3, format='csc') * anchor_weights

d_anchors = np.array([[0,0,0], [0, 0, 200], [50, 0, 120], [0, 0, 50]]) * anchor_weights

# prepare iterative solve for edges
new_vertices = vertices.copy()

for _ in range (iteration_count):
    
    H = sp.lil_matrix((len(edges), L.shape[0]))
    d_lengths = np.zeros((len(edges), 3))
    
    for idx, (i, j) in enumerate(edges):
        # original length
        original = np.sum((vertices[i] - vertices[j])**2)
    
        # current difference:
        v = new_vertices[i] - new_vertices[j]
        new_len = np.sum((v)**2)
    
        # create lengths
        H[idx, i] = length_weights
        H[idx, j] = -length_weights
    
        # compute rhs
        d_lengths[idx, :] = v * (original/new_len) * length_weights

    H3D = sp.kron(H, I3, format='csc')
    A = sp.vstack([L3D, A_anchor3D, H3D], format='csc')
    rhs = np.concatenate([delta_3D, d_anchors.flatten(), d_lengths.flatten()])
    
    new_vertices = spla.lsqr(A, rhs)[0].reshape(-1, 3)

draw(Adjacency, vertices, new_vertices)

In [ ]:
anchor_weights = 10.0

L, Adjacency, vertices = generate_laplacian(15, 200, 20)

# I3 is the 3x3 identity matrix
I3 = sp.eye(3, format='csc')

L3D = sp.kron(L, I3, format='csc')

delta_3D = L3D @ vertices.flatten()

A_anchor = sp.lil_matrix((4, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, 1] = 1
A_anchor[2, -1] = 1
A_anchor[3, -2] = 1

A_anchor3D = sp.kron(A_anchor, I3, format='csc') * anchor_weights

d_anchors = np.array([[0, 0, 0], [20, 0, 0], [120, 0, 90], [120, 0, 110]]) * anchor_weights

A = sp.vstack([L3D, A_anchor3D], format='csc')
rhs = np.concatenate([delta_3D, d_anchors.flatten()])

new_vertices = spla.lsqr(A, rhs)[0].reshape(-1, 3)

draw(Adjacency, vertices, new_vertices)

## Laplacian Editing with Local Transformations

we defined the **original energy functional** as:

$$
E(V') = \sum_{i=1}^{n} \| \delta_i - L v'_i \|^2 + \sum_{i \in C} \| v'_i - u_i \|^2
$$

The previous energy function does not account for **rotations and scaling** in deformations. To address this, we introduce **local per-vertex transformations** $T_i$ that adjust the Laplacian coordinates:

$$
E(V') = \sum_{i=1}^{n} \| T_i \delta_i - L v'_i \|^2 + \sum_{i \in C} \| v'_i - u_i \|^2
$$

where:
- $T_i$ is a **local transformation matrix** for vertex $i$.
- $T_i$ modifies $\delta_i$ before comparing it to the reconstructed Laplacian $L v'_i$.

This formulation ensures that local details undergo **rigid and isotropic transformations**, preventing distortion.

### Constructing the Matrix $A$ and Computing $T_i$

The transformation matrix $T_i$ is **computed from the neighborhood of vertex $i$** using a least-squares approach.

#### Step 1: Building the System Matrix $A_i$
To estimate $T_i$, we construct a system:

$$
A_i x = b_i
$$

where:
- $A_i$ contains the positions of vertex $i$ and its neighbors.
- $b_i$ contains the **new deformed positions** of $i$ and its neighbors.
- $x = (s, h_1, h_2, h_3, t_x, t_y, t_z)$ are the unknown transformation parameters.

For a vertex $i$ with neighbors $j \in N_i$, $A_i$ is structured as:

$$
A_i =
\begin{bmatrix}
v_{jx} & 0 & v_{jz} & -v_{jy} & 1 & 0 & 0 \\
v_{jy} & -v_{jz} & 0 & v_{jx} & 0 & 1 & 0 \\
v_{jz} & v_{jy} & -v_{jx} & 0 & 0 & 0 & 1 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots
\end{bmatrix}
$$

where each row corresponds to a neighbor $j$ of $i$.

#### Step 2: Solving for $T_i$
We solve the **least-squares system**:

$$
x = (A_i^T A_i)^{-1} A_i^T b_i
$$

which gives the optimal **scale, rotation, and translation** parameters.

Using these parameters, we construct $T_i$:

$$
T_i =
\begin{bmatrix}
s & -h_3 & h_2 & t_x \\
h_3 & s & -h_1 & t_y \\
-h_2 & h_1 & s & t_z \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

#### Step 3: Update L

Because $T_i$ depends on $V'$ we can compute the $(A_i^T A_i)^{-1} A_i^T$ part, compute the $T_i \cdot \delta_i$ and remove it from $L$.  
Now, we solve for 
$$ (L-T\delta)V' = 0 $$

In [ ]:
anchor_weights = 10.0

L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

# I3 is the 3x3 identity matrix
I3 = sp.eye(3, format='csc')
L3D = sp.kron(L, I3, format='lil')

# compute rotation invariance
for i in range(Adjacency.shape[0]):
    indices = np.concatenate([[i], Adjacency[i,:].nonzero()[1]])
    C = np.zeros([indices.shape[0] * 3, 7])
    for j, k in enumerate(indices):
        C[j*3+0] = [vertices[k, 0], 0, vertices[k, 2], -vertices[k, 1], 1, 0, 0]
        C[j*3+1] = [vertices[k, 1], -vertices[k, 2], 0, vertices[k, 0], 0, 1, 0]
        C[j*3+2] = [vertices[k, 2], vertices[k, 1], -vertices[k, 0], 0, 0, 0, 1]

    C_pinv = np.linalg.pinv(C)

    s = C_pinv[0]
    h = C_pinv[1:4]
    t = C_pinv[4:7]
    
    T_delta = np.vstack([
         delta[i,0]*s    - delta[i,1]*h[2] + delta[i,2]*h[1],
         delta[i,0]*h[2] + delta[i,1]*s    - delta[i,2]*h[0],
        -delta[i,0]*h[1] + delta[i,1]*h[0] + delta[i,2]*s   ,
    ])

    for j, k in enumerate(indices):
        L3D[i*3+0, k*3:k*3+3] -= T_delta[0, j*3:j*3+3]
        L3D[i*3+1, k*3:k*3+3] -= T_delta[1, j*3:j*3+3]
        L3D[i*3+2, k*3:k*3+3] -= T_delta[2, j*3:j*3+3]
    
A_anchor = sp.lil_matrix((4, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, 1] = 1
A_anchor[2, -1] = 1
A_anchor[3, -2] = 1

A_anchor3D = sp.kron(A_anchor, I3, format='csc') * anchor_weights

d_anchors = np.array([[0, 0, 0], [20, 0, 0], [120, 0, 80], [120, 0, 110]]) * anchor_weights

A = sp.vstack([L3D, A_anchor3D], format='csc')
rhs = np.concatenate([np.zeros([L3D.shape[0]]), d_anchors.flatten()])

new_vertices = spla.lsqr(A, rhs)[0].reshape(-1, 3)

draw(Adjacency, vertices, new_vertices)

### Add Edge Constraints

In [ ]:
iteration_count = 50
anchor_weights = 5
edges = [(i,i+1) for i in range(0,30,2)]

L, Adjacency, vertices = generate_laplacian(15, 200, 20)

delta = L @ vertices

# I3 is the 3x3 identity matrix
I3 = sp.eye(3, format='csc')
L3D = sp.kron(L, I3, format='lil')

# compute rotation invariance
for i in range(Adjacency.shape[0]):
    indices = np.concatenate([[i], Adjacency[i,:].nonzero()[1]])
    C = np.zeros([indices.shape[0] * 3, 7])
    for j, k in enumerate(indices):
        C[j*3+0] = [vertices[k, 0], 0, vertices[k, 2], -vertices[k, 1], 1, 0, 0]
        C[j*3+1] = [vertices[k, 1], -vertices[k, 2], 0, vertices[k, 0], 0, 1, 0]
        C[j*3+2] = [vertices[k, 2], vertices[k, 1], -vertices[k, 0], 0, 0, 0, 1]

    C_pinv = np.linalg.pinv(C)

    s = C_pinv[0]
    h = C_pinv[1:4]
    t = C_pinv[4:7]
    
    T_delta = np.vstack([
         delta[i,0]*s    - delta[i,1]*h[2] + delta[i,2]*h[1],
         delta[i,0]*h[2] + delta[i,1]*s    - delta[i,2]*h[0],
        -delta[i,0]*h[1] + delta[i,1]*h[0] + delta[i,2]*s   ,
    ])

    for j, k in enumerate(indices):
        L3D[i*3+0, k*3:k*3+3] -= T_delta[0, j*3:j*3+3]
        L3D[i*3+1, k*3:k*3+3] -= T_delta[1, j*3:j*3+3]
        L3D[i*3+2, k*3:k*3+3] -= T_delta[2, j*3:j*3+3]
    
A_anchor = sp.lil_matrix((4, L.shape[0]))
A_anchor[0, 0] = 1
A_anchor[1, 1] = 1
A_anchor[2, -1] = 1
A_anchor[3, -2] = 1

A_anchor3D = sp.kron(A_anchor, I3, format='csc') * anchor_weights
d_anchors = np.array([[0, 0, 0], [20, 0, 0], [120, 0, 80], [120, 0, 100]]) * anchor_weights


# prepare iterative solve for edges
new_vertices = vertices.copy()

for _ in range (iteration_count):
    
    H = sp.lil_matrix((len(edges), L.shape[0]))
    d_lengths = np.zeros((len(edges), 3))
    
    for idx, (i, j) in enumerate(edges):
        # original length
        original = np.linalg.norm(vertices[i] - vertices[j])
    
        # current difference:
        v = new_vertices[i] - new_vertices[j]
        new_len = np.linalg.norm(v)
    
        # create lengths
        H[idx, i] = 1
        H[idx, j] = -1
    
        # compute rhs
        d_lengths[idx, :] = v * (original/new_len)

    H3D = sp.kron(H, I3, format='csc')
    A = sp.vstack([L3D, A_anchor3D, H3D], format='csc')
    rhs = np.concatenate([np.zeros([L3D.shape[0]]), d_anchors.flatten(), d_lengths.flatten()])
    
    new_vertices = spla.lsqr(A, rhs)[0].reshape(-1, 3)

draw(Adjacency, vertices, new_vertices)

# Define Functions

In [ ]:
def update_L_rotation_invariance(L3D, Adjacency, vertices, delta):
    
    # compute rotation invariance
    for i in range(Adjacency.shape[0]):
        indices = np.concatenate([[i], Adjacency[i,:].nonzero()[1]])
        C = np.zeros([indices.shape[0] * 3, 7])
        for j, k in enumerate(indices):
            C[j*3+0] = [vertices[k, 0], 0, vertices[k, 2], -vertices[k, 1], 1, 0, 0]
            C[j*3+1] = [vertices[k, 1], -vertices[k, 2], 0, vertices[k, 0], 0, 1, 0]
            C[j*3+2] = [vertices[k, 2], vertices[k, 1], -vertices[k, 0], 0, 0, 0, 1]
    
        C_pinv = np.linalg.pinv(C)
    
        s = C_pinv[0]
        h = C_pinv[1:4]
        t = C_pinv[4:7]
        
        T_delta = np.vstack([
             delta[i,0]*s    - delta[i,1]*h[2] + delta[i,2]*h[1],
             delta[i,0]*h[2] + delta[i,1]*s    - delta[i,2]*h[0],
            -delta[i,0]*h[1] + delta[i,1]*h[0] + delta[i,2]*s   ,
        ])
    
        for j, k in enumerate(indices):
            L3D[i*3+0, k*3:k*3+3] -= T_delta[0, j*3:j*3+3]
            L3D[i*3+1, k*3:k*3+3] -= T_delta[1, j*3:j*3+3]
            L3D[i*3+2, k*3:k*3+3] -= T_delta[2, j*3:j*3+3]

def deform(L, Adjacency, vertices, anchor_indices, anchor_positions, edge_indices=None, edge_lengths=None, lock_y_anchor_indices = None, use_rotation_invariance=True, hard_constraints=False, max_iteration=10, anchor_weight = 5.0, length_weight = 5.0):

    # I3 is the 3x3 identity matrix
    I3 = sp.eye(3, format='csc')
    L3D = sp.kron(L, I3, format='lil')

    delta = None
    
    # modify L and b if we use rotation invariance
    if use_rotation_invariance:
        update_L_rotation_invariance(L3D, Adjacency, vertices, L @ vertices)
        delta = np.zeros([L3D.shape[0]])
    else:
        delta = L3D @ vertices.flatten()

    L3D = L3D.tocsc()

    # compute anchor constraints
    if hard_constraints:
        anchor_weight = 1.0
        
    A_anchor = sp.lil_matrix((anchor_indices.shape[0], L.shape[0]))
    A_anchor[np.arange(anchor_indices.shape[0]), anchor_indices] = 1
    A_anchor3D = sp.kron(A_anchor, I3, format='csc') * anchor_weight
    d_anchors = anchor_positions.flatten() * anchor_weight

    if lock_y_anchor_indices is not None:
        A_y_anchor = sp.lil_matrix((lock_y_anchor_indices.shape[0], L3D.shape[0]))
        A_y_anchor[np.arange(lock_y_anchor_indices.shape[0]), lock_y_anchor_indices*3+1] = 1
        A_anchor3D = sp.vstack((A_anchor3D, A_y_anchor))
        d_anchors = np.concatenate((d_anchors, vertices[lock_y_anchor_indices, 1]))

    # prepare iterative solve for edges
    new_vertices = vertices.copy()
    
    iteration_count = 1
    edge_constraint_count = 0
    if edge_indices is not None:
        iteration_count = max_iteration
        edge_constraint_count = edge_indices.shape[0]
        
        H = sp.lil_matrix((edge_constraint_count, L.shape[0]))
        H[np.arange(edge_constraint_count), edge_indices[:, 0]] = length_weight
        H[np.arange(edge_constraint_count), edge_indices[:, 1]] = -length_weight

        H3D = sp.kron(H, I3, format='csc')
    else:
        H3D = sp.lil_matrix((0, L3D.shape[0]))

    Laplacian = L3D
    if hard_constraints:
        Laplacian = L3D.transpose().dot(L3D)
        
    zero_block = sp.csc_matrix((A_anchor3D.shape[0] + H3D.shape[0], A_anchor3D.shape[0] + H3D.shape[0]))
    
    for _ in range (iteration_count):
        
        d_lengths = np.zeros((edge_constraint_count, 3))

        if edge_constraint_count > 0:
        
            # current difference:
            d0 = new_vertices[edge_indices[:, 0]] - new_vertices[edge_indices[:, 1]]
            norm_d0_sq = np.sum((d0)**2, axis=1)
                
            # compute rhs
            d_lengths = d0 * (length_weight * edge_lengths / norm_d0_sq)[:, np.newaxis]

        # solve :::
        
        if hard_constraints:
            # lagrange multiplier
            A_constraint = sp.vstack([A_anchor3D, H3D], format='csc')
            A = sp.bmat([
                [Laplacian, A_constraint.transpose()],
                [A_constraint, zero_block]
            ], format='csc')
            
            new_vertices = spla.spsolve(
                A, 
                np.concatenate([
                    L3D.transpose().dot(delta.flatten()), 
                    d_anchors,
                    d_lengths.flatten()
                ])
            )[:vertices.shape[0]*3].reshape(-1, 3)

        else:
            # least square
            A = sp.vstack([Laplacian, A_anchor3D, H3D], format='csc')
            rhs = np.concatenate([delta, d_anchors, d_lengths.flatten()])
            
            new_vertices = spla.lsqr(A, rhs)[0].reshape(-1, 3)
    return new_vertices

In [ ]:
def render(use_edge=True, use_rotation_invariance=True, hard_constraints=False, iteration=3):
    L, Adjacency, vertices = generate_laplacian(15, 200, 20)
    
    # anchors = np.array([0,1,-1,-2], dtype=np.int32)
    # d_anchors = np.array([[0, 0, 0], [20, 0, 0], [120, 0, 80], [120, 0, 100]])
    anchors = np.array([0,-1,12,5], dtype=np.int32)
    d_anchors = np.array([[0,0,0], [0, 0, 200], [50, 0, 120], [0, 0, 50]])
    
    edges = np.array([[i,i+1] for i in range(4,26,2)], dtype=np.int32)
    #edges = np.array(((1,3),(3,5),(5,7),(7,9)), dtype=np.int32)
    edges_Lengths = np.sum((vertices[edges[:, 1]] - vertices[edges[:, 0]])**2, axis=1)

    if use_edge == False:
        edges = None
        
    new_vertices = deform(L, Adjacency, vertices, anchors, d_anchors, edges, edges_Lengths, use_rotation_invariance = use_rotation_invariance, hard_constraints=hard_constraints, max_iteration=iteration)
    draw(Adjacency, vertices, new_vertices)

interact(
    render,
    iteration=widgets.IntSlider(min=1, max=30)
)
viewer

# Laplacian Animation Deformation

In [ ]:
# load the character
character = viewer.import_usd_asset('AnimLabSimpleMale.usd')
# character.add_bone('LeftHeel', np.array([1,0,0,0]), np.array([9.2,0,-12]), 'LeftFoot')
# character.add_bone('LeftBall', np.array([1,0,0,0]), np.array([14.5,0,8.22]), 'LeftFoot')
# character.add_bone('RightHeel', np.array([1,0,0,0]), np.array([-9.2,0,-12]), 'RightFoot')
# character.add_bone('RightBall', np.array([1,0,0,0]), np.array([-14.5,0,8.22]), 'RightFoot')

# left_heel = character.bone_index('LeftHeel')
# left_ball = character.bone_index('LeftBall')
# right_heel = character.bone_index('RightHeel')
# right_ball = character.bone_index('RightBall')
left_foot = character.bone_index('LeftFoot')
right_foot = character.bone_index('RightFoot')
left_toe = character.bone_index('LeftToe')
right_toe = character.bone_index('RightToe')

In [ ]:
animmap = lab.AnimMapper(character, keep_translation=False, root_motion=True, match_effectors=True, local_offsets={'Hips':[0, 2, 0]})
animation = lab.import_bvh('../../resources/lafan1/bvh/walk1_subject2.bvh', anim_mapper=animmap)

animation.quats = animation.quats[1641:, ...]
animation.pos = animation.pos[1641:, ...]

frame_count = animation.quats.shape[0]
bone_count = character.bone_count()

In [ ]:
# compute the foot contacts
gquats, gpos = lab.utils.quat_fk(animation.quats, animation.pos, animation.parents)

foot_tags = np.zeros([frame_count, 4], dtype=np.bool_)
foot_tags[:, :2], foot_tags[:, 2:] = lab.utils.extract_feet_contacts(gpos, [left_foot, left_toe], [right_foot, right_toe],  0.16)

# smooth out a little the signal ( we keep the signal on if it switch off for one or 2 frames )
for frame in range(2, frame_count-2):
    for c in range(4):
        foot_tags[frame, c] = foot_tags[frame, c] or (foot_tags[frame-2, c] and foot_tags[frame+2, c])
    
for frame in range(1, frame_count-1):
    for c in range(4):
        foot_tags[frame, c] = foot_tags[frame, c] or (foot_tags[frame-1, c] and foot_tags[frame+1, c])

In [ ]:
def render(frame):
    
    anim = animation
    p = (anim.pos[frame,...])
    q = (anim.quats[frame,...])
        
    a = lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(p[0])
    
    viewer.begin_shadow()
    viewer.draw(character, a)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a)    
    viewer.end_display()

    viewer.disable(depth_test=True)
   
    viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    viewer.draw_lines(character.world_skeleton_lines(a))
    
    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max=frame_count-1)
)
viewer

## Convert Animation As Laplacian

In [ ]:
def get_animation(animation, frame_start, frame_end):
    fcount = frame_end-frame_start

    bone_names = [
        'Hips',
        'Spine',
        'LeftUpLeg',
        'LeftLeg',
        'LeftFoot',
        'LeftToe',
        'RightUpLeg',
        'RightLeg',
        'RightFoot',
        'RightToe'
    ]

    bone_indices = np.array([character.bone_index(n) for n in bone_names], dtype=np.uint8)

    bcount = bone_indices.shape[0]

    # get the positions in the animation
    _, gpos = lab.utils.quat_fk(animation.quats[frame_start:frame_end,...], animation.pos[frame_start:frame_end, ...], animation.parents)
    vertices = gpos[:, bone_indices, :].reshape(-1, 3)

    # compute skeleton adjacency so we can compute the real adjacency
    SkelAdjacency = sp.lil_matrix((bcount, bcount))
    for i, index in enumerate(bone_indices):
        if animation.parents[index] >= 0 and animation.parents[index] in bone_indices:
            j = int(np.argwhere(bone_indices == animation.parents[index])[0][0])
            SkelAdjacency[i, j] = 1
            SkelAdjacency[j, i] = 1

    # duplicate skeleton adjacency for each frame
    Adjacency = sp.kron(sp.eye(fcount), SkelAdjacency).tolil()

    return vertices, Adjacency, bone_names, bcount, fcount


def connect_frames(Adjacency, bcount, fcount):
    # connect frames
    for i in range(fcount-1):
        for j in range(bcount):
            Adjacency[i*bcount+j, (i+1)*bcount+j] += 1
            Adjacency[(i+1)*bcount+j, (i)*bcount+j] += 1


def get_laplacian(Adjacency):
    return compute_degree_matrix(Adjacency) - Adjacency


In [ ]:
vertices, Adjacency, bone_names, bcount, fcount = get_animation(animation, 100, 169)
skeleton_adjacency = Adjacency.copy()
connect_frames(Adjacency, bcount, fcount)
L = get_laplacian(Adjacency)

anchors = np.arange(0,10)
d_anchors = vertices[anchors]

anchors = np.concatenate((anchors, vertices.shape[0] + np.arange(-10,0)))
d_anchors = np.concatenate((d_anchors, d_anchors+np.array([[0,0,-300]])))

#build edge with feet constraint
count = 0
edges = np.zeros([fcount*2, 2], dtype=np.int32)
for k, i in enumerate(range(100, 169-1)):
    if foot_tags[i, 1]:
        edges[count, 0] = k*bcount + bone_names.index('LeftToe')
        edges[count, 1] = (k+1)*bcount + bone_names.index('LeftToe')
        count += 1
    if foot_tags[i, 3]:
        edges[count, 0] = k*bcount + bone_names.index('RightToe')
        edges[count, 1] = (k+1)*bcount + bone_names.index('RightToe')
        count += 1
    if foot_tags[i, 0]:
        edges[count, 0] = k*bcount + bone_names.index('LeftFoot')
        edges[count, 1] = (k+1)*bcount + bone_names.index('LeftFoot')
        count += 1
    if foot_tags[i, 2]:
        edges[count, 0] = k*bcount + bone_names.index('RightFoot')
        edges[count, 1] = (k+1)*bcount + bone_names.index('RightFoot')
        count += 1
edges = edges[:count, :]

#add leg lengths
Leg_edges = np.zeros([fcount, 6, 2], dtype=np.int32)
Leg_edges[:, 0, 0] = np.arange(bone_names.index('LeftUpLeg'), vertices.shape[0], bcount)
Leg_edges[:, 0, 1] = np.arange(bone_names.index('LeftLeg'), vertices.shape[0], bcount)
Leg_edges[:, 1, 0] = np.arange(bone_names.index('LeftLeg'), vertices.shape[0], bcount)
Leg_edges[:, 1, 1] = np.arange(bone_names.index('LeftFoot'), vertices.shape[0], bcount)
Leg_edges[:, 2, 0] = np.arange(bone_names.index('RightUpLeg'), vertices.shape[0], bcount)
Leg_edges[:, 2, 1] = np.arange(bone_names.index('RightLeg'), vertices.shape[0], bcount)
Leg_edges[:, 3, 0] = np.arange(bone_names.index('RightLeg'), vertices.shape[0], bcount)
Leg_edges[:, 3, 1] = np.arange(bone_names.index('RightFoot'), vertices.shape[0], bcount)
Leg_edges[:, 4, 0] = np.arange(bone_names.index('RightFoot'), vertices.shape[0], bcount)
Leg_edges[:, 4, 1] = np.arange(bone_names.index('RightToe'), vertices.shape[0], bcount)
Leg_edges[:, 5, 0] = np.arange(bone_names.index('LeftFoot'), vertices.shape[0], bcount)
Leg_edges[:, 5, 1] = np.arange(bone_names.index('LeftToe'), vertices.shape[0], bcount)
edges = np.vstack((edges, Leg_edges.reshape(-1, 2)))

edges_Lengths = np.sum((vertices[edges[:, 1]] - vertices[edges[:, 0]])**2, axis=1)

#build feet height constraint
feet_height = np.concatenate((np.arange(bone_names.index('LeftToe'),vertices.shape[0], bcount), np.arange(bone_names.index('RightToe'),vertices.shape[0], bcount)))

# result
new_vertices = None

def render(use_edge=True, use_feet_height=True, use_rotation_invariance=False, hard_constraints=False, iteration=3):

    global new_vertices
    
    edges_to_use = None
    if use_edge:
        edges_to_use = edges

    feet_to_use = None
    if use_feet_height:
        feet_to_use = feet_height
    
    new_vertices = deform(L.copy(), Adjacency, vertices, anchors, d_anchors, edges_to_use, edges_Lengths, feet_to_use, use_rotation_invariance = use_rotation_invariance, hard_constraints=hard_constraints, max_iteration=iteration)
    draw(skeleton_adjacency, vertices, new_vertices, edges_to_use)

interact(
    render,
    iteration=widgets.IntSlider(min=1, max=100)
)
viewer

## Convert Deformed Laplacian Into Animation

In [ ]:
def compute_animation(quats, pos, vertices, vertices_bone_count, vertices_bone_names):
    fcount = quats.shape[0]
    
    # remove root node animation
    gquats, gpos = lab.utils.quat_fk(quats, pos, animation.parents)
    gquats[:, 0, :], gpos[:, 0 , :] = np.array([1,0,0,0])[np.newaxis, :], np.array([0,0,0])[np.newaxis, :]
    quats, pos = lab.utils.quat_ik(gquats, gpos, animation.parents)

    # move hips to the vertices position
    pos[:, character.bone_index('Hips'), :] = vertices[vertices_bone_names.index('Hips')::vertices_bone_count, :]
    gquats, gpos = lab.utils.quat_fk(quats, pos, animation.parents)

    # compute hips orientation
    z = vertices[vertices_bone_names.index('LeftUpLeg')::vertices_bone_count, :] - vertices[vertices_bone_names.index('RightUpLeg')::vertices_bone_count, :]
    x = vertices[vertices_bone_names.index('Spine')::vertices_bone_count, :] - vertices[vertices_bone_names.index('Hips')::vertices_bone_count, :]
    z /= np.linalg.norm(z, axis=1, keepdims=True)
    x /= np.linalg.norm(x, axis=1, keepdims=True)
    y = np.linalg.cross(z, x)
    z = np.linalg.cross(x, y)
    m = np.einsum("ijk->ikj", np.hstack((x,y,z)).reshape(-1, 3, 3))
    quats[:, character.bone_index('Hips'), :] = lab.utils.m3x3_to_quat(m)
    gquats, gpos = lab.utils.quat_fk(quats, pos, animation.parents)

    # compute foot
    def _foot_target(foot_name, toe_name):
        y = lab.utils.quat_mul_vec(gquats[:, character.bone_index(foot_name), :], np.array([[0,1,0]]))
        x = vertices[vertices_bone_names.index(toe_name)::vertices_bone_count, :] - vertices[vertices_bone_names.index(foot_name)::vertices_bone_count, :]
        x /= np.linalg.norm(x, axis=1, keepdims=True)
        z = np.linalg.cross(x, y)
        y = np.linalg.cross(z, x)
        return lab.utils.m3x3_to_quat(np.einsum("ijk->ikj", np.hstack((x,y,z)).reshape(-1, 3, 3))), vertices[vertices_bone_names.index(foot_name)::vertices_bone_count, :]

        
    target_foot_q, target_foot_p = np.zeros([fcount, 2, 4]), np.zeros([fcount, 2, 3])
    target_foot_q[:, 0, : ], target_foot_p[:, 0, : ]= _foot_target('LeftFoot', 'LeftToe')
    target_foot_q[:, 1, : ], target_foot_p[:, 1, : ]= _foot_target('RightFoot', 'RightToe')
    
    return lab.utils.limb_ik(
        quats, 
        pos, 
        animation.parents, 
        animation.bones, 
        target_foot_q,
        target_foot_p
    )

In [ ]:
quats, pos = compute_animation(animation.quats[100:169,...], animation.pos[100:169,...], new_vertices, bcount, bone_names)

def render(frame):
    
    p = (pos[frame,...])
    q = (quats[frame,...])
        
    a = lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(p[0])
    
    viewer.begin_shadow()
    viewer.draw(character, a)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a)    
    viewer.end_display()

    viewer.disable(depth_test=True)

    draw(skeleton_adjacency, None, new_vertices, final_draw=False)
   
    viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    viewer.draw_lines(character.world_skeleton_lines(a))

    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max=fcount-1)
)
viewer

In [ ]:
walk_straight = lab.Anim(quats.copy(), pos.copy(), pos[0].copy(), animation.parents, animation.bones)
walk_straight = lab.animation.compute_root_motion(walk_straight)

walk_straight.pos[:, 0, :] -= walk_straight.pos[0, 0, :]


## Curve the path of locomotion

In [ ]:
def curve(L, angle, x, y, x_offset=0):

    if np.abs(angle)<0.01 :
        return x, y
        
    R = L / angle  # Compute radius of curvature
    center_x, center_y = -R, 0  # Circle center
    theta = y / R  # Angle along the arc
    
    R += x #offset of radius depending on x
    R += x_offset
    
    # Compute position
    x_prime = center_x + R * np.cos(theta)
    y_prime = center_y + R * np.sin(theta)
    
    return x_prime, y_prime

In [ ]:
def render(angle=0, shift=0):
    
    viewer.begin_shadow()
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()  
    viewer.end_display()

    cnt = 100
    vertices = np.zeros([cnt, 2, 3])
    vertices[:, 0, 0] = 20
    vertices[:, 1, 0] = -20
    vertices[:, 0, 2] = np.linspace(0, -300, cnt)
    vertices[:, 1, 2] = np.linspace(0, -300, cnt)

    for i in range(cnt):
        vertices[i, 0, 0], vertices[i, 0, 2] = curve(300, angle, vertices[i, 0, 0], vertices[i, 0, 2], shift)
        vertices[i, 1, 0], vertices[i, 1, 2] = curve(300, angle, vertices[i, 1, 0], vertices[i, 1, 2], shift)
    
    viewer.disable(depth_test=True)
    
    buf_size = 512
    v = np.zeros([buf_size,3], dtype=np.float32)
    vi = 0
    for i in range(cnt-1):
        v[vi] = vertices[i, 0, :]
        v[vi+1] = vertices[i+1, 0, :]
        vi += 2
    viewer.draw_lines(v)
    vi = 0
    for i in range(cnt-1):
        v[vi] = vertices[i, 1, :]
        v[vi+1] = vertices[i+1, 1, :]
        vi += 2
    viewer.draw_lines(v, color=np.array([1,1,0], dtype=np.float32))

    viewer.execute_commands()
    
interact(
    render, 
    angle = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi),
    shift = widgets.FloatSlider(value=0, min=-100, max=100)
)
viewer

In [ ]:
vertices, Adjacency, bone_names, bcount, fcount = get_animation(walk_straight, 0, fcount)
skeleton_adjacency = Adjacency.copy()
connect_frames(Adjacency, bcount, fcount)
L = get_laplacian(Adjacency)
anchors = np.arange(0,vertices.shape[0])

def render(angle=0, use_edge=True, use_feet_height=True, iteration=3):

    global new_vertices

    d_anchors = vertices.copy()
    for i in range(d_anchors.shape[0]):
        d_anchors[i, 0], d_anchors[i, 2] = curve(300, angle, vertices[i, 0], vertices[i, 2], vertices[i, 1]*.05*-angle)
    
    edges_to_use = None
    if use_edge:
        edges_to_use = edges

    feet_to_use = None
    if use_feet_height:
        feet_to_use = feet_height
    
    new_vertices = deform(L.copy(), Adjacency, vertices, anchors, d_anchors, edges_to_use, edges_Lengths, feet_to_use, use_rotation_invariance = False, hard_constraints=False, max_iteration=iteration, anchor_weight=5)
    draw(skeleton_adjacency, vertices, new_vertices, edges_to_use)

interact(
    render,
    angle=widgets.FloatSlider(min=-np.pi*2, max=np.pi*2),
    iteration=widgets.IntSlider(min=1, max=100)
)
viewer

In [ ]:
quats, pos = compute_animation(walk_straight.quats, walk_straight.pos, new_vertices, bcount, bone_names)

def render(frame):
    
    p = (pos[frame,...])
    q = (quats[frame,...])
        
    a = lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(p[0])
    
    viewer.begin_shadow()
    viewer.draw(character, a)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a)    
    viewer.end_display()

    viewer.disable(depth_test=True)

    draw(skeleton_adjacency, None, new_vertices, final_draw=False)
   
    viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    viewer.draw_lines(character.world_skeleton_lines(a))

    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max=fcount-1)
)
viewer